In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import tensorflow as tf
import numpy as np
from numpy.linalg import norm
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from sklearn.neighbors import NearestNeighbors
import tensorflow_text as text
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from transformers import DistilBertTokenizer, DistilBertModel
import torch
from tqdm import tqdm

from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.cluster import KMeans


file_path = "/kaggle/input/clothing-fit-dataset-for-size-recommendation/renttherunway_final_data.json"
data = pd.read_json(file_path, lines=True)

df = pd.DataFrame(data)

2024-05-03 16:50:11.446703: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-03 16:50:11.446862: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-03 16:50:11.623917: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
df.dropna(inplace=True)

In [3]:
df['height_inches'] = df['height'].apply(lambda x: int(x.split("'")[0]) * 12 + int(x.split("'")[1].replace('"', '')))
df['weight_lbs'] = df['weight'].apply(lambda x: int(x.replace('lbs', '')))

product_df = df.groupby('item_id').agg({
    'review_text': lambda x: ' '.join(x),
    'review_summary': lambda x: ' '.join(x),
    'height_inches': 'mean',
    'weight_lbs': 'mean',
    'age': 'mean',
    'rating': 'mean', 
    
}).reset_index()

product_df['category'] = df.groupby('item_id').apply(lambda x: ', '.join(x['category'].unique())).reset_index(drop=True)
product_df['bodytype'] = df.groupby('item_id').apply(lambda x: ', '.join(x['body type'].unique())).reset_index(drop=True)
product_df['rented_for']= df.groupby('item_id').apply(lambda x: ', '.join(x['rented for'].unique())).reset_index(drop=True)

product_df.rename(columns={'height_inches': 'avg_height', 'weight_lbs': 'avg_weight', 'age': 'avg_age', 'rating': 'avg_rating'}, inplace=True)

/tmp/ipykernel_33/3774860903.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  product_df['category'] = df.groupby('item_id').apply(lambda x: ', '.join(x['category'].unique())).reset_index(drop=True)
/tmp/ipykernel_33/3774860903.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  product_df['bodytype'] = df.groupby('item_id').apply(lambda x: ', '.join(x['body type'].unique())).reset_index(drop=True)
/tm

In [4]:
product_df.head()

,item_id,review_text,review_summary,avg_height,avg_weight,avg_age,avg_rating,category,bodytype,rented_for
0,123373,The dress was beautiful and very comfortable. ...,It was a very beautiful black tie wedding I lo...,65.297189,141.108434,35.178715,8.871486,gown,"straight & narrow, full bust, hourglass, athle...","wedding, formal affair, party, other, vacation..."
1,123793,"Fit great, super flattering Limited range of a...",Stunning gown. Wore this for heart ball and re...,65.050222,132.706056,31.671344,9.528804,gown,"athletic, pear, petite, hourglass, straight & ...","formal affair, wedding, other, party, date, work"
2,124204,"This dress is a ""WOW."" It steals the show, sp...",When I opened the RTR box and saw these sequin...,65.090909,137.614876,33.800000,9.325620,dress,"petite, hourglass, full bust, athletic, straig...","formal affair, party, other, wedding, vacation..."
3,124553,Loved the dress. A little short in the front ...,Lovely wedding guest dress Fabulous! Added a b...,65.450000,139.655556,33.211111,8.527778,dress,"full bust, pear, petite, athletic, straight & ...","wedding, other, work, formal affair, date, par..."
4,125424,The dress would have been perfect in a size 10...,I didn't wear it but it would have been perfec...,65.680672,136.683473,32.742297,9.025210,dress,"straight & narrow, hourglass, athletic, pear, ...","wedding, party, other, formal affair, date, va..."


In [5]:
model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model.trainable = False

model = tf.keras.Sequential([
    model,
    GlobalMaxPooling2D()
])

def extract_features(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    result = model.predict(preprocessed_img).flatten()
    normalized_result = result / norm(result)
    return normalized_result

import os
from PIL import Image

directory = '/kaggle/input/image-data/data'
files = os.listdir(directory)

files.sort()
images = []

for filename in files:
    if filename.endswith('.png'):
        image_path = os.path.join(directory, filename)
        images.append(image_path)

l=[]
for img_path in images:
    features = extract_features(img_path, model)
    l.append(features)
    print(f"Embeddings for {img_path}: {features}")

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Embeddings for /kaggle/input/image-data/data/0.png: [0.01712174 0.01402907 0.02034878 ... 0.03794208 0.03313183 0.00849118]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
Embeddings for /kaggle/input/image-data/data/1.png: [0.01836157 0.00555044 0.02447125 ... 0.06053744 0.01911784 0.07746886]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
Embeddings for /kaggle/input/image-data/data/10.png: [0.00547957 0.01466619 0.00290963 ... 0.02941033 0.01456989 0.01300918]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
Embeddings for /kaggle/input/image-data/data/11.png: [0.03816883 0.02351381 0.02150268 ... 0.02229217 0.012012   0.03022448]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
Embeddings for /kaggle/input/image-data/data/12.png: [0.01583838 0.02573645 0.00715203 ... 0.03039647 0.00199451 0.01716576]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step
Embeddings for /kaggle/input/image-data/data/13.png: [0.02106054 0.03070302 0.00744762 ... 0.04

In [6]:
len(l)

100

In [7]:
new_user_image_embedding = extract_features('/kaggle/input/new-user/Screenshot 2024-05-03 131439.png', model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step


In [8]:
rt=pd.read_csv('/kaggle/input/reviewtext/Image_Review.csv')
rt=pd.read_csv('/kaggle/input/image-review/Image_Review.csv')

In [9]:
rt.head()

,Product_id,Rating,Review
0,1229740,10,"super cute, 8 was super tight so i wore the 10..."
1,2475563,10,Fit like a glove. I wasn't tugging up the dres...
2,125465,8,Ordered the 4 and 6. 4 was a little tight in ...
3,753788,8,The draping was unexpected and I might have pr...
4,123373,10,The dress is very elegant and comfortable. It ...


In [10]:

def add_word_to_groups(review):
    words_per_group = 20
    review_words = review.split()
    for i in range(0, len(review_words), words_per_group):
        group = review_words[i:i + words_per_group]
        group_index = i // words_per_group
        if group_index < len(words):
            group.append(words[group_index])
    return ' '.join(review_words)

def add_rating_info(review, rating):
    return f"{review} This got a rating - {rating}"

words = ["gown", "sheath", "top", "jacket", "Dress"]
rt['Review'] = rt['Review'].apply(add_word_to_groups)
rt['Review'] = rt.apply(lambda row: add_rating_info(row['Review'], row['Rating']), axis=1)

In [11]:
r=rt['Rating']
r=r.to_numpy()

In [12]:
type(r)

numpy.ndarray

In [13]:
type(l)

list

In [14]:
image_embeddings=np.asarray(l)

In [15]:
def text_embeddings_from_reviews(review_texts):
    text_embeddings_list = []
    vectorizer = TfidfVectorizer()
    for review_text in review_texts:
        tfidf_embedding = vectorizer.fit_transform([review_text])
        text_embeddings_list.append(tfidf_embedding)

    text_embeddings_array = np.array(text_embeddings_list)

    return text_embeddings_array


In [16]:
review_texts = rt['Review'].tolist()
# text_embeddings = text_embeddings_from_reviews(review_texts)

In [17]:
text_embeddings = text_embeddings_from_reviews(review_texts)
type(text_embeddings)

numpy.ndarray

In [18]:
text_embeddings.shape

(100,)

In [19]:
padded = []
max_columns = max(embedding.shape[1] for embedding in text_embeddings)
for embedding in text_embeddings:
    padding = np.zeros((1, max_columns - embedding.shape[1]))
    embedding = np.hstack((embedding.toarray(), padding))
    padded.append(embedding)

In [20]:
review_embeddings = np.vstack(padded)
review_embeddings.shape 

(100, 114)

In [21]:
image_embeddings.shape

(100, 2048)

In [22]:

X_train, X_test, y_train, y_test = train_test_split(image_embeddings, review_embeddings, test_size=0.2, random_state=42)

model = keras.Sequential([
    layers.Dense(256, activation='relu', input_shape=(2048,)),  
    layers.Dense(256, activation='relu'),                       
    layers.Dense(128, activation='relu'),                      
    layers.Dense(114)                                           
])

model.compile(optimizer='adam', loss='mse')

history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test))

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)


Epoch 1/50


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 231ms/step - loss: 0.0088 - val_loss: 0.0075
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.0073 - val_loss: 0.0064
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.0060 - val_loss: 0.0057
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.0053 - val_loss: 0.0051
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.0047 - val_loss: 0.0046
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0042 - val_loss: 0.0042
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.0038 - val_loss: 0.0041
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.0035 - val_loss: 0.0040
Epoch 9/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 0.0034 - val_loss: 0.0039
Epoch 10/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 0.0033 - val_loss: 0.0039
Epoch 11/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0032 - val_loss: 0.0038
Epoch 12/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.0033 - val_loss: 0.0039
Epoch 13/50

In [23]:
input_layer = keras.Input(shape=(2048,))

x = layers.Dense(256, activation='relu')(input_layer)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(100, activation='relu')(x)
output_layer = layers.Dense(114)(x)

model = keras.Model(inputs=input_layer, outputs=output_layer)

model.compile(optimizer='adam', loss='mse')

history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test))

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)
_ = model.predict(X_train[:1])

penultimate_output_model = keras.Model(inputs=model.input, outputs=model.layers[-2].output)
latent_embeddings_t = penultimate_output_model.predict(image_embeddings)

Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 210ms/step - loss: 0.0088 - val_loss: 0.0081
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.0080 - val_loss: 0.0072
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.0071 - val_loss: 0.0066
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.0064 - val_loss: 0.0060
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.0058 - val_loss: 0.0054
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.0052 - val_loss: 0.0050
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.0047 - val_loss: 0.0047
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.0044 - val_loss: 0.0045
Epoch 9/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.0041 - val_loss: 0.0042
Epoch 10/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.0038 - val_loss: 0.0041
Epoch 11/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.0036 - val_loss: 0.0040
Epoch 12/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.0035 - val_loss: 0.0040


In [24]:

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
distilbert_model = DistilBertModel.from_pretrained('distilbert-base-uncased')

def text_embeddings_from_reviews(review_texts):
    text_embeddings_list = []
    for review_text in tqdm(review_texts, desc="Generating Text Embeddings"):
        inputs = tokenizer(review_text, return_tensors='pt')  # Use 'pt' for PyTorch tensors
        with torch.no_grad():
            outputs = distilbert_model(**inputs)
        last_hidden_state = outputs.last_hidden_state
        pooled_output = last_hidden_state[:, 0, :384]  # Select the first 384 elements
        pooled_output = np.squeeze(pooled_output.numpy())  # Squeeze the tensor before appending
        text_embeddings_list.append(pooled_output)

    text_embeddings_array = np.array(text_embeddings_list)
    return text_embeddings_array

textb_embeddings = text_embeddings_from_reviews(review_texts)
print(textb_embeddings.shape)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Generating Text Embeddings: 100%|██████████| 100/100 [00:09<00:00, 10.59it/s]

(100, 384)


In [25]:
X_train, X_test, y_train, y_test = train_test_split(image_embeddings, textb_embeddings, test_size=0.2, random_state=42)

input_layer = keras.Input(shape=(2048,))

x = layers.Dense(256, activation='relu')(input_layer)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(100, activation='relu')(x)
output_layer = layers.Dense(384)(x)

model = keras.Model(inputs=input_layer, outputs=output_layer)

model.compile(optimizer='adam', loss='mse')

history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test))

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)
_ = model.predict(X_train[:1])

penultimate_output_model = keras.Model(inputs=model.input, outputs=model.layers[-2].output)
latent_embeddings_b = penultimate_output_model.predict(image_embeddings)

Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 209ms/step - loss: 0.1875 - val_loss: 0.1815
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 0.1818 - val_loss: 0.1714
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.1712 - val_loss: 0.1564
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.1561 - val_loss: 0.1427
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.1429 - val_loss: 0.1302
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.1301 - val_loss: 0.1138
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.1136 - val_loss: 0.0999
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.1001 - val_loss: 0.0860
Epoch 9/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0860 - val_loss: 0.0724
Epoch 10/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.0725 - val_loss: 0.0593
Epoch 11/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.0591 - val_loss: 0.0464
Epoch 12/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.0461 - val_loss: 0.0358


In [26]:
rt.head()

,Product_id,Rating,Review
0,1229740,10,"super cute, 8 was super tight so i wore the 10..."
1,2475563,10,Fit like a glove. I wasn't tugging up the dres...
2,125465,8,Ordered the 4 and 6. 4 was a little tight in t...
3,753788,8,The draping was unexpected and I might have pr...
4,123373,10,The dress is very elegant and comfortable. It ...


In [96]:
!pip install implicit

In [29]:
!pip install lightfm
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 5.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=464211 sha256=4e6bb30d905ca20bff6774dc620eb9840df791ccee51e40bec7d130546c96b5c
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


### Lightfm model with BPR loss

In [88]:
user_ids = list(range(100))
ratings = rt["Rating"]
product_ids = rt['Product_id']

combined_embeddings = np.concatenate((latent_embeddings_b, textb_embeddings), axis=1)

combined_sparse = csr_matrix(combined_embeddings)

train, test = random_train_test_split(combined_sparse, test_percentage=25, random_state=42)

model = LightFM(loss='bpr')

model.fit(train, epochs=25)

precision_at_k_score = precision_at_k(model, test, k=5).mean() 
print("Precision at K:", precision_at_k_score)

Precision at K: 0.85200006


In [31]:
def get_recommendations(model, user_id, user_features, item_features, k=5):
    scores = model.predict(user_id, np.arange(item_features.shape[0]), user_features=user_features, item_features=item_features)
    top_indices = np.argsort(-scores)[:k]
    
    return top_indices

user_id = 15
top_indices = get_recommendations(model, user_id, None, combined_sparse)

print("Recommended Product IDs for User", user_id, ":")
for index in top_indices:
    print(product_ids[index])


Recommended Product IDs for User 15 :
2062332
2281848
2245994
2591874
2626811


### A model from scratch to use BPR 

In [91]:
import numpy as np
from scipy.sparse import csr_matrix
from tqdm import tqdm

# Hyperparameters
learning_rate = 0.01
num_epochs = 25
num_factors = 100
regularization = 0.01

# Convert data to sparse matrix
combined_sparse = csr_matrix(combined_embeddings)

# Initialize user and item matrices randomly
num_users, num_items = combined_sparse.shape
user_matrix = np.random.normal(size=(num_users, num_factors))
item_matrix = np.random.normal(size=(num_items, num_factors))

# Training BPR
for epoch in range(num_epochs):
    # Shuffle interactions
    interactions = combined_sparse.nonzero()
    indices = np.arange(len(interactions[0]))
    np.random.shuffle(indices)
    
    # Iterate over interactions
    for idx in tqdm(indices, desc=f"Epoch {epoch + 1}/{num_epochs}", total=len(interactions[0])):
        # Extract user and item index from the interaction
        user_idx = interactions[0][idx]
        item_idx = interactions[1][idx]
        
        # Sample a negative item that the user hasn't interacted with
        negative_item_idx = np.random.choice(np.where(combined_sparse[user_idx].toarray() == 0)[1])
        
        # Compute difference in scores
        pos_score = np.dot(user_matrix[user_idx], item_matrix[item_idx])
        neg_score = np.dot(user_matrix[user_idx], item_matrix[negative_item_idx])
        score_diff = pos_score - neg_score
        
        # Update user and item matrices
        user_matrix[user_idx] += learning_rate * (score_diff * (item_matrix[item_idx] - item_matrix[negative_item_idx]) - regularization * user_matrix[user_idx])
        item_matrix[item_idx] += learning_rate * (score_diff * user_matrix[user_idx] - regularization * item_matrix[item_idx])
        item_matrix[negative_item_idx] += learning_rate * (-score_diff * user_matrix[user_idx] - regularization * item_matrix[negative_item_idx])


Epoch 1/25:   1%|          | 417/42000 [00:00<00:09, 4164.78it/s]/tmp/ipykernel_33/1011847001.py:41: RuntimeWarning: overflow encountered in multiply
  user_matrix[user_idx] += learning_rate * (score_diff * (item_matrix[item_idx] - item_matrix[negative_item_idx]) - regularization * user_matrix[user_idx])
/tmp/ipykernel_33/1011847001.py:42: RuntimeWarning: overflow encountered in multiply
  item_matrix[item_idx] += learning_rate * (score_diff * user_matrix[user_idx] - regularization * item_matrix[item_idx])
/tmp/ipykernel_33/1011847001.py:43: RuntimeWarning: overflow encountered in multiply
  item_matrix[negative_item_idx] += learning_rate * (-score_diff * user_matrix[user_idx] - regularization * item_matrix[negative_item_idx])
Epoch 25/25: 100%|██████████| 42000/42000 [00:09<00:00, 4397.64it/s]


Recommended Product IDs for User 15 :
1229740


KeyError: 330

In [95]:
# Example of getting recommendations for a user
user_id = 15
top_indices = get_recommendations(user_id)
print("Recommended Product IDs for User", user_id, ":")
max_index = min(max(top_indices), len(product_ids) - 1)  # Ensure index is within bounds
for index in top_indices:
    if index <= max_index:  # Check if index is within bounds
        print(product_ids[index])
    else:
        print("Index out of range:", index)


Recommended Product IDs for User 15 :
1229740
Index out of range: 330
Index out of range: 329
Index out of range: 328
Index out of range: 327


## For New User

In [97]:
def get_text_embeddings(text):
    vectorizer = TfidfVectorizer()
    text_embeddings = vectorizer.fit_transform([text])
    return text_embeddings.toarray()

new_text = "Party high cost"
new_embedding = get_text_embeddings(new_text)

padding = np.zeros((1, max_columns - new_embedding.shape[1]))
new_embedding_padded = np.hstack((new_embedding, padding))

In [98]:
kmeans = KMeans(n_clusters=3).fit(review_embeddings)

new_embedding_cluster = kmeans.predict(new_embedding_padded.reshape(1, -1))
similar_indices = np.where(kmeans.labels_ == new_embedding_cluster)[0]

for index in similar_indices[:3]:
    print(index)

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


6
13
17
